In [1]:
import os
import bigframes.pandas as bf
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import time
import datetime

In [2]:
bf.options.bigquery.project = 'credit-business-on-demand-9fd5'

In [34]:
query_tpv_interno = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.tpv_interno`"""
df_tpv_interno = bf.read_gbq(query_tpv_interno)
base_tpv_interno = df_tpv_interno.to_pandas()

In [ ]:
query_doc = f""" SELECT * FROM `dataplatform-prd.credit_negocios.document_notfound`"""
df_doc = bf.read_gbq(query_doc)
base_doc = df_doc.to_pandas()

In [ ]:
query_tam_acq = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.tam_acquirer`"""
df_tam_acq = bf.read_gbq(query_tam_acq)
base_tam_acq = df_tam_acq.to_pandas()

In [ ]:
query_tam = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.tam`"""
df_tam = bf.read_gbq(query_tam)
base_tam = df_tam.to_pandas()

In [ ]:
base_doc_tam = base_doc.merge(base_tam,how='left',left_on=['DocumentNumber'],right_on=['cpf_cnpj'])

In [ ]:
base_doc_tam_na = base_doc_tam.groupby('DocumentNumber', as_index=False).agg(
    is_na=('tam_id', lambda x: x.isna().sum())).reset_index()

In [ ]:
base_doc_tam_na['is_na'].value_counts()

In [ ]:
base_tam_doc = base_doc_tam.dropna(subset=['tam_id']).merge(base_tam_acq,how='left',on=['tam_id'])

In [ ]:
base_tam_doc_ = base[base['document'].isin(base_tam_doc['cpf_cnpj'].unique().tolist())]

In [ ]:
base_tam_doc_['cliente_micro'] = np.where(base_tam_doc_['client_type'] == 'Micro',1,0)

In [ ]:
base_tam_doc_agg = base_tam_doc_.groupby('document').agg(is_micro=('cliente_micro','sum')).reset_index()

In [ ]:
base_tam_doc_agg[base_tam_doc_agg['is_micro'] >0 ]

In [ ]:
base_tam_doc_agg.shape

In [ ]:
base = base_tpv_interno[(base_tpv_interno['document'].isin(base_doc['DocumentNumber'].unique().tolist()))&(base_tpv_interno['stone_acquirer_active_30d'] == True)]
base.sort_values(by=['document','reference_month'],inplace=True)

In [ ]:
base['is_cliente_novo'] = np.where(base['client_type'] == 'Cliente novo',1,0)

In [ ]:
base_agg = base.groupby('document').agg(clientes_novos = ('is_cliente_novo','sum')).reset_index()

In [ ]:
base_agg_ = base.groupby('document').agg(unicos_tipos = ('client_type','nunique')).reset_index()

In [ ]:
base_agg_['unicos_tipos'].value_counts()

In [ ]:
base_agg['clientes_novos'].value_counts()

In [ ]:
# Ultima atualização: 28/10/2025

In [3]:
query_cadastrados_stone = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.clientes_ativos_2210`"""
df_cadastrados_stone = bf.read_gbq(query_cadastrados_stone)
base_cadastrados_stone = df_cadastrados_stone.to_pandas()

In [4]:
query_cadastrados_ton = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.clientes_ton_2810`"""
df_cadastrados_ton = bf.read_gbq(query_cadastrados_ton)
base_cadastrados_ton = df_cadastrados_ton.to_pandas()
   

In [9]:
base_cadastrados_stone['DocumentNumber_len'] = base_cadastrados_stone['DocumentNumber'].str.len()

In [11]:
base_cadastrados_ton['document_len'] = base_cadastrados_ton['document'].str.len()

In [12]:
doc_stone = base_cadastrados_stone[(base_cadastrados_stone['DocumentNumber_len'] == 14)&(~base_cadastrados_stone['DocumentNumber'].isin(base_tpv_master['document'].unique().tolist()))][['DocumentNumber']]

In [13]:
doc_ton = base_cadastrados_ton[(base_cadastrados_ton['document_len'] == 14)&(~base_cadastrados_ton['document'].isin(base_tpv_master['document'].unique().tolist()))][['document']]

In [19]:
doc_stone.rename(columns={'DocumentNumber':'document'},inplace=True)

In [23]:
doc_notfound = pd.concat([doc_stone,doc_ton],axis=0)

In [ ]:
base_tpv_interno

In [37]:
base_tpv_interno[base_tpv_interno['document'].isin(doc_notfound['document'].unique().tolist())]

,reference_month,document,ton_acquirer_active_30d,stone_acquirer_active_30d,tpv_cross_ton_stone,credit_card_tpv,ton_acquirer_trx_credit,stone_acquirer_trx_credit
18,2025-10-01,43295446000101,True,False,2836.760000000,0.0,43,0
34,2025-10-01,21359787000152,False,False,0E-9,0.0,0,0
58,2025-10-01,62957047000103,False,False,0E-9,0.0,0,0
62,2025-10-01,50809269000191,True,False,4614.000000000,0.0,29,0
67,2025-10-01,34276547000136,False,False,0E-9,0.0,0,0
...,...,...,...,...,...,...,...,...
16530297,2025-10-01,14779411000131,False,False,0E-9,0.0,0,0
16530320,2025-10-01,42630047000189,False,False,0E-9,0.0,0,0
16530328,2025-10-01,12016751000185,False,False,0E-9,0.0,0,0
16530342,2025-10-01,44708412000156,False,False,0E-9,0.0,0,0


In [33]:
base_tpv_master[base_tpv_master['origem_tpv'] == 'StoneCo Only'].isna().sum()

document                               0
reference_month                        0
tpv_stone                              0
tpv_total                              0
share_tpv_debit                     2076
share_tpv_credit                    2076
share_tpv_chip_present           5557882
share_tpv_chip_not_present       5557882
share_tpv_recurring              5557882
share_tpv_online                 5557882
average_ticket_size              5557882
average_spend_per_card           5557882
average_transactions_per_card    5557882
origem_tpv                             0
dtype: int64

In [39]:
base_tpv_interno[base_tpv_interno['document'].isin(base_tpv_master[base_tpv_master['origem_tpv'] == 'StoneCo Only']['document'].unique().tolist())]

,reference_month,document,ton_acquirer_active_30d,stone_acquirer_active_30d,tpv_cross_ton_stone,credit_card_tpv,ton_acquirer_trx_credit,stone_acquirer_trx_credit
23,2025-10-01,58869901000102,False,False,0E-9,0.0,0,0
35,2025-10-01,34915217000143,False,False,0E-9,0.0,0,0
46,2025-10-01,21087688000169,False,False,0E-9,0.0,0,0
54,2025-10-01,23350641000171,False,True,41731.000000000,0.0,0,40
55,2025-10-01,48227608000180,False,False,0E-9,0.0,0,0
...,...,...,...,...,...,...,...,...
16530300,2025-10-01,29171814000170,False,False,0E-9,0.0,0,0
16530332,2025-10-01,34757221000120,False,False,0E-9,0.0,0,0
16530335,2025-10-01,13811726000156,True,True,5982.990000000,0.0,30,0
16530338,2025-10-01,21138883000170,False,False,-1496.060000000,0.0,0,2


In [40]:
base_tpv_interno['qtd'] = base_tpv_interno['ton_acquirer_trx_credit'] +  base_tpv_interno['stone_acquirer_trx_credit']

In [ ]:
base_ativos_30d[base_ativos_30d['document'].isin(base_cadastrados_ton['DocumentNumber'].unique().tolist())]['stone_acquirer_active_30d'].value_counts()

In [ ]:
base_cadastrados_stone['DocumentNumber_len'] = base_cadastrados_stone['DocumentNumber'].str.len()

In [ ]:
base_cadastrados_stone['DocumentNumber_len'].value_counts()

In [ ]:
ls_doc_stone = base_cadastrados_stone[base_cadastrados_stone['DocumentNumber_len'] == 14]['DocumentNumber'].unique().tolist()

In [ ]:
base_cadastrados_ton['document_len'] = base_cadastrados_ton['document'].str.len()

In [ ]:
base_cadastrados_ton['document_len'].value_counts()

In [ ]:
ls_doc_ton = base_cadastrados_ton[base_cadastrados_ton['document_len'] == 14]['document'].unique().tolist()

In [ ]:
from google.cloud import bigquery

#%% Exportacao
client = bigquery.Client(project = 'dataplatform-prd')

table_name = 'credit_negocios.document_notfound'

job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")
job = client.load_table_from_dataframe(df, table_name, job_config = job_config)

In [ ]:
base_tpv_master[base_tpv_master['document'].isin(ls_doc_stone)]['document'].nunique()

In [ ]:
base_tpv_master[base_tpv_master['document'] == '22744280000185']

In [ ]:
base_tpv_master[base_tpv_master['document'].isin(ls_doc_ton)]['document'].nunique()

In [ ]:
base_tpv_master[(base_tpv_master['document'].isin(ls_doc_stone))|(base_tpv_master['document'].isin(ls_doc_ton))]['document'].nunique()

In [ ]:
base_tpv_master['document'].nunique()

In [ ]:
base_ativos_30d['document_len'] = base_ativos_30d['document'].str.len()

In [ ]:
base_ativos_30dbase_ativos_30d['document_len']

In [ ]:
base_ativos_30d['document'].count()

In [ ]:
ls_doc_stone_ativos = base_ativos_30d[(base_ativos_30d['stone_acquirer_active_30d'] == True)&(base_ativos_30d['document_len'] == 14)]['document'].unique().tolist()

In [ ]:
base_tpv_master[(base_tpv_master['document'].isin(ls_doc_stone_ativos))]['document'].nunique()

In [ ]:
query_ativos_30d = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.clientes_ativos_30d_2810`"""
df_ativos_30d = bf.read_gbq(query_ativos_30d)
base_ativos_30d = df_ativos_30d.to_pandas()

In [ ]:
base_cadastrados_stone[['DocumentNumber']].to_csv('base_cadastrados_stone.csv',index=False)

In [ ]:
base_cadastrados_ton.to_csv('base_cadastrados_ton.csv',index=False)

In [ ]:
base_ativos_30d[base_ativos_30d['ton_acquirer_active_30d'] == True][['document']].to_csv('base_ativos_30d_ton.csv',index=False)

In [ ]:
base_ativos_30d[base_ativos_30d['stone_acquirer_active_30d'] == True][['document']].to_csv('base_ativos_30d_stone.csv',index=False)

In [8]:
query_tpv_master = f""" SELECT * FROM `dataplatform-prd.economic_research.sbca_monthly_metrics`"""
df_tpv_master = bf.read_gbq(query_tpv_master)
base_tpv_master = df_tpv_master.to_pandas()

In [ ]:
query_estabelecimentos_municipios_rf = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.estabelecimentos_municipio_rf`"""
df_estabelecimentos_municipios_rf = bf.read_gbq(query_estabelecimentos_municipios_rf)
base_estabelecimentos_municipios_rf = df_estabelecimentos_municipios_rf.to_pandas()

In [ ]:
query_cnae_documentos = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.cnae_documentos`"""
df_cnae_documentos = bf.read_gbq(query_cnae_documentos)
base_cnae_documentos = df_cnae_documentos.to_pandas()

In [ ]:
base_tpv_master = base_tpv_master.merge(base_estabelecimentos_municipios_rf,how='left',left_on=['document'],right_on=['cpf_cnpj'])

In [ ]:
base_cnae_documentos['grupo_cnae_fiscal_principal'] = base_cnae_documentos['cnae_fiscal_principal'].astype(str).str[:2]

In [ ]:
base_cnae_documentos

In [ ]:
base_tpv_master = base_tpv_master.merge(base_cnae_documentos,how='left',left_on=['document'],right_on=['cpf_cnpj'])

In [ ]:
base_tpv_master.to_csv(r'base_tpv_master_completa.csv',index=False)

In [ ]:
base_tpv_master = base_tpv_master[base_tpv_master['reference_month'].dt.date >= datetime.date(2024,8,1)]

In [ ]:
base_tpv_master_avg_ticked_size = base_tpv_master.groupby(['document']).agg(avg_ticket_size_construido = ('average_ticket_size','mean')).reset_index()

In [ ]:
base_tpv_master_avg_ticked_size.isna().sum()

In [ ]:
base_tpv_master = base_tpv_master.merge(base_tpv_master_avg_ticked_size,how='left',on=['document'])

In [ ]:
base_tpv_master = base_tpv_master[~base_tpv_master['avg_ticket_size_construido'].isna()]

In [ ]:
base_tpv_master['avg_ticket_size_construido'] = pd.to_numeric(base_tpv_master['avg_ticket_size_construido'], errors='coerce')

In [ ]:
base_tpv_master['avg_ticket_size_construido'] = base_tpv_master['avg_ticket_size_construido'].astype('int')

In [ ]:
base_tpv_master['q'] = (base_tpv_master['tpv_total']/base_tpv_master['avg_ticket_size_construido'])

In [ ]:
base_tpv_master.loc[base_tpv_master['q'] == 0, 'q'] = np.nan

In [ ]:
base_tpv_master['total_tpv_credit'] = base_tpv_master['share_tpv_credit']*base_tpv_master['tpv_total']

In [ ]:
base_main = base_tpv_master[['document','q','reference_month','total_tpv_credit']]

In [ ]:
base_main = base_main.merge(base_estabelecimentos_municipios_rf,how='left',left_on=['document'],right_on=['cpf_cnpj'])

In [ ]:
base_main_agg = base_main.groupby(['reference_month','municipio']).agg(total_tpv_credit_mun=('total_tpv_credit','sum')).reset_index()

In [ ]:
base_main = base_main.merge(base_main_agg,how='left',on=['reference_month','municipio'])

In [ ]:
base_cnae_documentos['grupo_cnae_fiscal_principal'] = base_cnae_documentos['cnae_fiscal_principal'].astype(str).str[:2]

In [ ]:
base_main = base_main.merge(base_cnae_documentos,how='left',left_on=['document'],right_on=['cpf_cnpj'])

In [ ]:
base_main = base_main[['document', 'q', 'reference_month', 'total_tpv_credit',
       'municipio', 'total_tpv_credit_mun',
       'cnae_fiscal_principal', 'grupo_cnae_fiscal_principal']]

In [ ]:
base_main.to_csv('base_tpv_master_q.csv',index=False)

In [ ]:
query_estabelecimentos_municipios = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.estabelecimentos_municipio`"""
df_estabelecimentos_municipios = bf.read_gbq(query_estabelecimentos_municipios)
base_estabelecimentos_municipios = df_estabelecimentos_municipios.to_pandas()

In [ ]:
query_enderecos_documentos = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.endereco_documentos`"""
df_enderecos_documentos = bf.read_gbq(query_enderecos_documentos)
base_enderecos_documentos = df_enderecos_documentos.to_pandas()

In [ ]:
query_documentos_por_socio = f""" SELECT * FROM `dataplatform-nonprd.credit_negocios.documentos_por_socio`"""
df_documentos_por_socio = bf.read_gbq(query_documentos_por_socio)
base_documentos_por_socio = df_documentos_por_socio.to_pandas()

In [ ]:
base_tpv_master['total_tpv_credit'] = base_tpv_master['share_tpv_credit']*base_tpv_master['tpv_total']

In [ ]:
base_tpv_master.groupby(['reference_month']).agg(soma_credito=('total_tpv_credit','sum')).reset_index().sort_values(by=['reference_month']).plot(kind='line',x='reference_month')

In [ ]:
base_tpv_master['month'] = base_tpv_master['reference_month'].dt.month
base_tpv_master['year'] = base_tpv_master['reference_month'].dt.year

In [ ]:
# Juntando a base do TPV master com a localização dos documentos, para termos uma medida de faturamento total na cidade

In [ ]:
base_tpv_master = base_tpv_master.merge(base_estabelecimentos_municipios_rf,how='left',left_on=['document'],right_on=['cpf_cnpj'])

In [ ]:
base_tpv_master_agg = base_tpv_master.groupby(['month','year','municipio']).agg(total_tpv_credit_mun=('total_tpv_credit','sum')).reset_index()

In [ ]:
base_tpv_master = base_tpv_master.merge(base_tpv_master_agg,how='left',on=['month','year','municipio'])

In [ ]:
# Merge com a informação CNAE dos documentos

In [ ]:
base_cnae_documentos['grupo_cnae_fiscal_principal'] = base_cnae_documentos['cnae_fiscal_principal'].astype(str).str[:2]

In [ ]:
base_tpv_master = base_tpv_master.merge(base_cnae_documentos,how='left',left_on=['document'],right_on=['cpf_cnpj'])

In [ ]:
# Merge com a informação de concorrência (mesmo CNAE/municipio)

In [ ]:
base_estabelecimentos_municipios['month'] = base_estabelecimentos_municipios['ano_mes'].str[-2:].astype(int)
base_estabelecimentos_municipios['year'] = base_estabelecimentos_municipios['ano_mes'].str[:4].astype(int)

In [ ]:
base_tpv_master = base_tpv_master.merge(base_estabelecimentos_municipios,how='left',left_on=['grupo_cnae_fiscal_principal','municipio','month','year'],right_on=['cnae_grupo','municipio','month','year'])

In [ ]:
# Merge com a base de informação sobre os sócios 

In [ ]:
base_tpv_master = base_tpv_master.merge(base_documentos_por_socio,how='left',left_on=['document'],right_on=['cpf_cnpj'])

In [ ]:
base_tpv_master[['media_cnpjs_por_socio','qtd_socios_no_mes']] = base_tpv_master[['media_cnpjs_por_socio','qtd_socios_no_mes']].fillna(1)

In [ ]:
base_tpv_master = base_tpv_master[['document', 'reference_month', 'tpv_stone', 'tpv_total',
       'share_tpv_debit', 'share_tpv_credit', 'share_tpv_chip_present',
       'share_tpv_chip_not_present', 'share_tpv_recurring', 'share_tpv_online',
       'average_ticket_size', 'average_spend_per_card',
       'average_transactions_per_card', 'origem_tpv', 'month', 'year',
       'total_tpv_credit', 'municipio', 'total_tpv_credit_mun',
       'grupo_cnae_fiscal_principal','total_estabelecimentos',
       'media_cnpjs_por_socio', 'qtd_socios_no_mes']]

In [ ]:
# Filtro para considerar a partir de Agosto de 2024

In [ ]:
base_tpv_master = base_tpv_master[(base_tpv_master['month'] >= 8)|(base_tpv_master['year'] >= 2025)]

In [ ]:
base_tpv_master[['log_total_tpv_credit','log_total_tpv_credit_mun']] = np.log(base_tpv_master[['total_tpv_credit','total_tpv_credit_mun']])

In [ ]:
base_tpv_master['log_total_tpv_credit_lag'] = (base_tpv_master.groupby('document')['log_total_tpv_credit'].shift(1))

In [ ]:
base_tpv_master['municipio'] = base_tpv_master['municipio'].astype(str)

In [ ]:
base_tpv_master = base_tpv_master[(~np.isinf(base_tpv_master['log_total_tpv_credit']))&(~np.isinf(base_tpv_master['log_total_tpv_credit_lag']))]

In [ ]:
base_tpv_master.to_csv('base_tpv_master.csv',index=False)

In [ ]:
base_rodagem = base_tpv_master[(base_tpv_master['month'] == 1)&(base_tpv_master['year'] == 2025)]

In [ ]:
base_rodagem.shape

In [ ]:
base_rodagem[['log_total_tpv_credit','log_total_tpv_credit_lag']].hist(bins=100)

In [ ]:
begin = time.time()

In [ ]:
model = smf.ols('log_total_tpv_credit ~ log_total_tpv_credit_lag + log_total_tpv_credit_mun + media_cnpjs_por_socio + qtd_socios_no_mes + C(municipio) + C(grupo_cnae_fiscal_principal)', data=base_rodagem).fit()

In [ ]:
end = time.time()

In [ ]:
print(model.summary())

In [ ]:
print(end - begin)

In [ ]:
# Analise dos residuos 

In [ ]:
model_results = model.fittedvalues.reset_index()
model_results.rename(columns={0:'log_total_tpv_credit_pred'},inplace=True)

In [ ]:
base_rodagem.reset_index(inplace=True)

In [ ]:
base_resultados = base_rodagem.merge(model_results,on=['index'])

In [ ]:
base_resultados['residuo'] = base_resultados['log_total_tpv_credit'] - base_resultados['log_total_tpv_credit_pred']

In [ ]:
base_resultados['residuo'].hist(bins=100)

In [ ]:
base_resultados['residuo_pct'] = np.exp(base_resultados['residuo']) - 1

In [ ]:
base_resultados.to_csv('vencedores_03_25.csv',index=False)